<a href="https://colab.research.google.com/github/SadafShafi/Sadaf-Avatar/blob/main/Conversational_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install --upgrade tensorflow

In [ ]:
!pip install pydub

In [ ]:
!pip install transformers

In [ ]:
!pip install ffmpeg-python

In [ ]:
!pip install sentencepiece

In [15]:
!pip install TensorFlowTTS 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 128 kB 5.0 MB/s 
     |████████████████████████████████| 3.5 MB 4.1 MB/s 
     |████████████████████████████████| 214 kB 8.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.4 MB 42.9 MB/s 
     |████████████████████████████████| 235 kB 59.3 MB/s 
     |████████████████████████████████| 3.1 MB 43.1 MB/s 
     |████████████████████████████████| 1.1 MB 54.0 MB/s 
     |████████████████████████████████| 1.7 MB 51.1 MB/s 
     |████████████████████████████████| 458.3 MB 9.2 kB/s 
     |████████████████████████████████| 132 kB 51.9 MB/s 
     |████████████████████████████████| 14.8 MB 40.5 MB/s 
     |████████████████████████████████| 20.2 MB 300 kB/s 
     |████████████████████████████████| 11.8 MB 46.9 MB/s 
     |███████████████

In [ ]:
!pip install -U transformers
!pip install sentencepiece

In [ ]:
# !pip install git+https://github.com/repodiac/german_transliterate.git#egg=german_transliterate 

In [ ]:
!pip install happytransformer

In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
from IPython.display import Audio

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]
  print("playing your answer")
  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [2]:
audio, sr = get_audio()

playing your answer


In [ ]:
# !pip install transformers

In [3]:
import librosa 
audio = audio.astype(np.float)
audio = librosa.resample(audio,orig_sr=sr,target_sr=16000)

In [7]:
# !pip install  "transformers==4.12.3" --upgrade

In [8]:
import transformers

In [9]:
# from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
# #  from datasets import load_dataset
# import torch

# # load model and tokenizer
# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
# model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
  
# #  # load dummy dataset and read soundfiles
# # #  ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")

# #  # tokenize
# input_values = processor(audio, return_tensors="pt", padding="longest").input_values  # Batch size 1

# #  # retrieve logits
# logits = model(input_values).logits

# #  # take argmax and decode
# predicted_ids = torch.argmax(logits, dim=-1)
# transcription = processor.batch_decode(predicted_ids)


In [10]:
# transcription[0]

In [ ]:
!pip install -U transformers
!pip install sentencepiece

In [4]:
import torch 
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration
# from datasets import load_dataset

model1 = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
processor1 = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")


ModuleNotFoundError: ignored

In [11]:
inputs = processor1(audio, sampling_rate=16000 , return_tensors="pt") 
generated_ids = model1.generate(inputs["input_features"], attention_mask=inputs["attention_mask"]) 
transcription = processor1.batch_decode(generated_ids) 
text = transcription[0] 
text

/usr/local/lib/python3.7/dist-packages/transformers/models/speech_to_text/modeling_speech_to_text.py:561: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  input_lengths = (input_lengths - 1) // 2 + 1
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


"there a cock will do his life now hello d'urber or you're goodly"

In [13]:
from happytransformer import HappyTextToText, TTSettings

happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")

args = TTSettings(num_beams=5, min_length=1)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

In [14]:
import numpy as np
import soundfile as sf
import yaml

import tensorflow as tf

from tensorflow_tts.inference import TFAutoModel
from tensorflow_tts.inference import AutoProcessor

# initialize fastspeech2 model.
fastspeech2 = TFAutoModel.from_pretrained("tensorspeech/tts-fastspeech2-ljspeech-en")


# initialize mb_melgan model
mb_melgan = TFAutoModel.from_pretrained("tensorspeech/tts-mb_melgan-ljspeech-en")


# inference
processor = AutoProcessor.from_pretrained("tensorspeech/tts-fastspeech2-ljspeech-en")

ModuleNotFoundError: ignored

In [ ]:
from transformers import pipeline
qa_model = pipeline("question-answering")
context = "I’m from a middle-class family of Bihar, I am Naresh Shukla. Nobody comes in this world, without the support of family and friends. Actually, whatever you will be, it is just because of your family. My father is a respectable businessman in our community.My mother is a doctor. They both love their occupation. That’s I have learned from my parents the value of time, honesty, hard work and commitment to the purpose.We are three brothers and sisters. Being the eldest I am the most liable from my brothers and sisters. I am wanted to guide and take care of my other siblings. We all are in the same school. Reading is my passion.I am a keen reader of novels and history books as I have a strong interest in Indian History and classical architecture. I love to read books that refer to the rich history and civilization of ancient India.  At my pre-childhood, I used to listen to stories from my grandmother, and this has a long-lasting effect on me."
# question = sentence
# ans = qa_model(question = question, context = context) 
# print("SimBot : " + ans["answer"]) 

RuntimeError: ignored

In [ ]:
for x in range(5):
  audio, sr = get_audio()
  audio = audio.astype(np.float)
  audio = librosa.resample(audio,orig_sr=sr,target_sr=16000)
  inputs = processor1(audio, sampling_rate=16000 , return_tensors="pt")
  generated_ids = model1.generate(inputs["input_features"], attention_mask=inputs["attention_mask"])

  transcription = processor1.batch_decode(generated_ids)
  text = transcription[0]
  print(text)
  result = happy_tt.generate_text(text, args=args)
  print("you >> ",result)
  sentence = result.text 
  text = sentence
  ans = qa_model(question = question, context = context) 
  print("Simbot >> ",ans["answer"])
  input_ids = processor.text_to_sequence(text)
  # fastspeech inference

  mel_before, mel_after, duration_outputs, _, _ = fastspeech2.inference(
      input_ids = tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
      speaker_ids = tf.convert_to_tensor([0], dtype=tf.int32),
      speed_ratios = tf.convert_to_tensor([1.0], dtype=tf.float32),
      f0_ratios = tf.convert_to_tensor([1.0], dtype=tf.float32),
      energy_ratios = tf.convert_to_tensor([1.0], dtype=tf.float32))
  # melgan inference
  # audio_before = mb_melgan.inference(mel_before)[0, :, 0]
  audio_after = mb_melgan.inference(mel_after)[0, :, 0]
  # save to file
  # sf.write('./audio_before.wav', audio_before, 22050, "PCM_16")
  sf.write('./audio_after.wav', audio_after, 22050, "PCM_16")
  print("play answer")
  wn = Audio("/content/audio_after.wav", rate=22050)
  display(wn)
  input()

playing your answer


/usr/local/lib/python3.7/dist-packages/transformers/models/speech_to_text/modeling_speech_to_text.py:561: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  input_lengths = (input_lengths - 1) // 2 + 1
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


hallo there


NameError: ignored

In [ ]:
sentence

'Starting it all into life.'

In [ ]:
help(Audio)

In [ ]:
!pip install happytransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 3.4 MB/s 
     |████████████████████████████████| 451 kB 50.5 MB/s 
     |████████████████████████████████| 115 kB 65.3 MB/s 
     |████████████████████████████████| 212 kB 65.6 MB/s 
     |████████████████████████████████| 127 kB 85.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from happytransformer import HappyTextToText, TTSettings

happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")

args = TTSettings(num_beams=5, min_length=1)

# Add the prefix "grammar: " before each input 
result = happy_tt.generate_text(text, args=args)
sentence = result.text
print(result.text) # This sentence has bad grammar.


In [ ]:
from transformers import pipeline
qa_model = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


RuntimeError: ignored

In [ ]:
context = "I’m from a middle-class family of Bihar, I am Naresh Shukla. Nobody comes in this world, without the support of family and friends. Actually, whatever you will be, it is just because of your family. My father is a respectable businessman in our community.My mother is a doctor. They both love their occupation. That’s I have learned from my parents the value of time, honesty, hard work and commitment to the purpose.We are three brothers and sisters. Being the eldest I am the most liable from my brothers and sisters. I am wanted to guide and take care of my other siblings. We all are in the same school. Reading is my passion.I am a keen reader of novels and history books as I have a strong interest in Indian History and classical architecture. I love to read books that refer to the rich history and civilization of ancient India.  At my pre-childhood, I used to listen to stories from my grandmother, and this has a long-lasting effect on me."

In [ ]:
from transformers import pipeline
qa_model = pipeline("question-answering")
context = "I’m from a middle-class family of Bihar, I am Naresh Shukla. Nobody comes in this world, without the support of family and friends. Actually, whatever you will be, it is just because of your family. My father is a respectable businessman in our community.My mother is a doctor. They both love their occupation. That’s I have learned from my parents the value of time, honesty, hard work and commitment to the purpose.We are three brothers and sisters. Being the eldest I am the most liable from my brothers and sisters. I am wanted to guide and take care of my other siblings. We all are in the same school. Reading is my passion.I am a keen reader of novels and history books as I have a strong interest in Indian History and classical architecture. I love to read books that refer to the rich history and civilization of ancient India.  At my pre-childhood, I used to listen to stories from my grandmother, and this has a long-lasting effect on me."
question = sentence
ans = qa_model(question = question, context = context) 
print("SimBot : " + ans["answer"]) 

In [ ]:
!pip install TensorFlowTTS 
!pip install git+https://github.com/repodiac/german_transliterate.git#egg=german_transliterate 

In [ ]:
import numpy as np
import soundfile as sf
import yaml

import tensorflow as tf

from tensorflow_tts.inference import TFAutoModel
from tensorflow_tts.inference import AutoProcessor

# initialize fastspeech2 model.
fastspeech2 = TFAutoModel.from_pretrained("tensorspeech/tts-fastspeech2-ljspeech-en")


# initialize mb_melgan model
mb_melgan = TFAutoModel.from_pretrained("tensorspeech/tts-mb_melgan-ljspeech-en")


# inference
processor = AutoProcessor.from_pretrained("tensorspeech/tts-fastspeech2-ljspeech-en")

In [ ]:
text = ans["answer"]
input_ids = processor.text_to_sequence(text)
# fastspeech inference

mel_before, mel_after, duration_outputs, _, _ = fastspeech2.inference(
    
    input_ids=tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
    speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
    speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
    f0_ratios =tf.convert_to_tensor([1.0], dtype=tf.float32),
    energy_ratios =tf.convert_to_tensor([1.0], dtype=tf.float32),
    
)

# melgan inference
audio_before = mb_melgan.inference(mel_before)[0, :, 0]
audio_after = mb_melgan.inference(mel_after)[0, :, 0]

# save to file
sf.write('./audio_before.wav', audio_before, 22050, "PCM_16")
sf.write('./audio_after.wav', audio_after, 22050, "PCM_16")

In [ ]:
from IPython.display import Audio
# signal = read_audio("/content/audio_after.wav").squeeze()
Audio("/content/audio_after.wav", rate=22050)

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
# from datasets import load_dataset
import torch
# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-large")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large")
# load dummy dataset and read soundfiles
# ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
input_features = processor(audio, return_tensors="pt").input_features 
# Generate logits
logits = model(input_features, decoder_input_ids = torch.tensor([[50258]])).logits 
# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)


In [ ]:
transcription

NameError: ignored